In [171]:
import dask.dataframe as dd
from dask_ml.preprocessing import DummyEncoder, Categorizer
from sklearn import datasets, linear_model
import math

Importing the data

In [172]:
raw = dd.read_csv("https://gist.githubusercontent.com/aromerovilla/7170e4ff45dd943af6a920d2f510cd0f/raw/b5732e3122ca9d0dec95717b62434e2b68e642b6/hour.csv")

In [173]:
raw.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 17 entries, instant to cnt
dtypes: object(1), float64(4), int64(12)

In [147]:
# def fix_types(df):
#     df['season'] = df['season'].astype('category')
#     df['yr'] = df['yr'].astype('category')
#     df['mnth'] = df['mnth'].astype('category')
#     df['hr'] = df['hr'].astype('category')
#     df['holiday'] = df['holiday'].astype('category')
#     df['weekday'] = df['weekday'].astype('category')
#     df['workingday'] = df['workingday'].astype('category')
#     df['weathersit'] = df['weathersit'].astype('category')
#     return df

In [174]:
raw.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

In [175]:
raw.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [176]:
selected = ['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', ]

In [177]:
categorizer = Categorizer(columns=selected)
categorizer = categorizer.fit(raw)

In [178]:
result = categorizer.transform(raw)

In [179]:
result.dtypes

instant          int64
dteday          object
season        category
yr            category
mnth          category
hr            category
holiday       category
weekday       category
workingday    category
weathersit    category
temp           float64
atemp          float64
hum            float64
windspeed      float64
casual           int64
registered       int64
cnt              int64
dtype: object

In [180]:
dummyencoder = DummyEncoder(columns=selected)
dummyencoder = dummyencoder.fit(result)

In [181]:
result = dummyencoder.transform(result)

In [182]:
result.dtypes

instant           int64
dteday           object
temp            float64
atemp           float64
hum             float64
windspeed       float64
casual            int64
registered        int64
cnt               int64
season_1          uint8
season_2          uint8
season_3          uint8
season_4          uint8
yr_0              uint8
yr_1              uint8
mnth_1            uint8
mnth_2            uint8
mnth_3            uint8
mnth_4            uint8
mnth_5            uint8
mnth_6            uint8
mnth_7            uint8
mnth_8            uint8
mnth_9            uint8
mnth_10           uint8
mnth_11           uint8
mnth_12           uint8
hr_0              uint8
hr_1              uint8
hr_2              uint8
                 ...   
hr_9              uint8
hr_10             uint8
hr_11             uint8
hr_12             uint8
hr_13             uint8
hr_14             uint8
hr_15             uint8
hr_16             uint8
hr_17             uint8
hr_18             uint8
hr_19           

## BASELINE

In [183]:
from dask.distributed import Client 
client = Client()

AttributeError: module 'dask' has no attribute 'config'